In [1]:
import sys
import sympy
from sympy import *
from sympy import Rational as frac
init_printing()

from memoize import memoize

vartheta, varphi = sympy.symbols('vartheta, varphi', real=True)
v, nu, delta, R_L = sympy.symbols('v, nu, delta, R_L', real=True)
Sigma_n, Sigma_ell, Sigma_lambda = sympy.symbols('Sigma_n, Sigma_ell, Sigma_lambda', real=True)
S_n, S_ell, S_lambda = sympy.symbols('S_n, S_ell, S_lambda', real=True)

nHat = Matrix([1,0,0])
lambdaHat = Matrix([0,1,0])
ellHat = Matrix([0,0,1])

NHat = Matrix([sin(vartheta)*cos(varphi), sin(vartheta)*sin(varphi), cos(vartheta)])
PHat = Matrix([sin(varphi), -cos(varphi), 0])
QHat = Matrix([cos(vartheta)*cos(varphi), cos(vartheta)*sin(varphi), -sin(vartheta)])

Sigma = Matrix([Sigma_n, Sigma_lambda, Sigma_ell])
S = Matrix([S_n, S_lambda, S_ell])

def hComplex(jComponent, kComponent):
    return sympy.trigsimp((nu/R_L)*( (jComponent.dot(PHat)*kComponent.dot(PHat) - jComponent.dot(QHat)*kComponent.dot(QHat))
                                    - sympy.I*(jComponent.dot(PHat)*kComponent.dot(QHat) + jComponent.dot(QHat)*kComponent.dot(PHat)) ) )

In [25]:
# Memoize (cache) this function for speed
@memoize
def sYlm(s, ell, m, vartheta, varphi):
    r = sympy.Symbol('r', integer=True)
    return sympy.simplify( sympy.sqrt(frac( factorial(ell+m)*factorial(ell-m),factorial(ell+s)*factorial(ell-s) )
                                      * (2*ell+1)/(4*sympy.pi) )
                           * exp(sympy.I*m*varphi)
                           * summation(binomial(ell-s,r) * binomial(ell+s,r+s-m) * (-1)**(ell-r-s-m)
                                       *sin(vartheta/2)**(2*ell-2*r-s+m) * cos(vartheta/2)**(2*r+s-m),
                                       (r,0,ell-s)) )
@memoize
def sYlmConjugate(s, ell, m, vartheta, varphi):
    r = sympy.Symbol('r', integer=True)
    return sympy.simplify( sympy.sqrt(frac( factorial(ell+m)*factorial(ell-m),factorial(ell+s)*factorial(ell-s) )
                                      * (2*ell+1)/(4*sympy.pi) )
                           * exp(-sympy.I*m*varphi)
                           * summation(binomial(ell-s,r) * binomial(ell+s,r+s-m) * (-1)**(ell-r-s-m)
                                       *sin(vartheta/2)**(2*ell-2*r-s+m) * cos(vartheta/2)**(2*r+s-m),
                                       (r,0,ell-s)) )
# Calculate these now, just so everything else will be fast
for ell in range(2,4+1):
    for m in range(-ell,ell+1):
        tmp = sYlm(-2,ell,m,vartheta,varphi)
        tmp = sYlmConjugate(-2,ell,m,vartheta,varphi)

The following are the spin contributions to the waveform from Kidder '95 (and the first-order nonspinning contribution, for comparison).

In [3]:
h0NS = 2*v**2*sympy.simplify(hComplex(lambdaHat, lambdaHat)-hComplex(nHat, nHat))

In [4]:
h2Ss = 2*v**4*hComplex(Sigma.cross(NHat), nHat)

In [5]:
h3SO = -2*v**5*sympy.simplify((5*S_ell + 3*delta*Sigma_ell)*hComplex(lambdaHat, lambdaHat)
                              - 6*(2*S_ell + delta*Sigma_ell)*hComplex(nHat, nHat))

In [6]:
h3Sp = -2*v**5*sympy.simplify(2*hComplex(lambdaHat, nHat.cross(S + delta*Sigma))
                              + hComplex(nHat, lambdaHat.cross(9*S + 5*delta*Sigma)) )

In [7]:
h3Spv = -2*v**5*sympy.simplify(2*NHat.dot(lambdaHat)*hComplex((S + delta*Sigma).cross(NHat), nHat)
                               + NHat.dot(nHat)*hComplex((S + delta*Sigma).cross(NHat), lambdaHat) )

In [39]:
def IntegrateFunctionOnTheSphere(func):
    return integrate( integrate( sympy.expand(func*sin(vartheta)), (varphi, 0, 2*pi)), (vartheta, 0, pi))
def IntegrateFunctionAgainstsYlm(func, ell, m, s=-2):
    func = sympy.expand(func*sYlmConjugate(s,ell,m,vartheta,varphi)*sin(vartheta))
    return integrate( integrate( func, (varphi, 0, 2*pi)), (vartheta, 0, pi))

This isn't going to work; `sympy` just can't handle the simple trig integrals necessary.

In [2]:
integrate(sin(vartheta)*cos(vartheta/2)**8, vartheta)

KeyboardInterrupt: 